Below are parameters for the Papercraft verifiable delay function (VDF).

In [2]:
from rok_estimator import *

In [3]:
mode = "C.L"

secpar = 128
kappa = 80
log_q = 62
rep = 57
t = 1024 * 3
T = t * rep
n_vdf = 14 

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 52.9,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel": 2,
    "log_beta_wit_inf": 0,
}

base = 2^7
ops_params = {
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = lambda base, n = True: [("bdecomp", {"base": base }), ("norm", {}), ("split", {"d": ops_params["d"]}), ("fold", {})] + ([("split", {"d": ops_params["d"]}), ("fold", {})] if n else [])

ops = small_loop + big_loop(2^7) + big_loop(2^9) + big_loop(2^7) + big_loop(2^7) + [("finish", {})]


sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show(brief=False)

print(f'VDF rounds T = {T}')


Ring parameters:
    conductor f: 24, degree phi: 8, modulus q: 2^62, beta_sis_2: 2^52.9000000000000
    SIS module rank n_sis: 220, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 62.00 B, |R_q^(n_sis)| = 13.32 KB
 
Subtractive set parameters:
    cardinality: 3
    forward ell_2 expansion factor gamma_2: 2^0
    inverse ell_2 expansion factor theta_2: 2^3
    forward ell_inf expansion factor gamma_inf: 2^1
    inverse ell_inf expansion factor theta_inf: 2^3
 
Relation:
    H * F * W = Y
Statement:
    H: n_compress x (n_commit + n_rel)
    F: (n_commit + n_rel) x wdim
    Y: n_compress x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2 measured in max column canonical ell_2-norm
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    n_compress = 222, n_commit = 220, n_rel = 2
    wdim = 2666496, rep = 57
    log_2-norm (real | extr) = (14 |14 ),

In [3]:
mode = "C.M"

secpar = 128
kappa = 80
log_q = 62
rep = 57
t = 512 * 3
T = t * rep
n_vdf = 14 

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 52.4,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel": 2,
    "log_beta_wit_inf": 0,
}

base = 2^7
ops_params = {
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = lambda base, n = True: [("bdecomp", {"base": base }), ("norm", {}), ("split", {"d": ops_params["d"]}), ("fold", {})] + ([("split", {"d": ops_params["d"]}), ("fold", {})] if n else [])

ops = small_loop + big_loop(2^8) + big_loop(2^9) + big_loop(2^7) + big_loop(2^9, false) + [("finish", {})]


sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

print(f'VDF rounds T = {T}')


Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  1333248 |  57 |        ( 14 | 14 )        |         (  0 | 14 )         | 72.47 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |  1333248 |  72 |        ( 14 | 14 )        |         (  0 | 14 )         | 91.55 MB |      (211.1 KB | 211.1 KB)      |         (2^-120 | 2^-120)         
 split     |   666624 | 144 |        ( 14 | 56*)        |         (  0 | 47 )         | 91.55 MB |      (1007. KB | 1.190 MB)      |         (2^-124 | 2^-120)         
 fold      |   666624 |  57 |        ( 22 | 49 )        |         (  9 | 44 )         | 362.4 MB |      (1007. KB | 1.190 MB)      |         (2^-83  | 2^-83 )         
 split     |   333312 | 114 |        ( 22 | 42 )        |         (  9 | 40 )         | 362.4 MB |      (797.2 KB | 1.968 MB)      |         (2

In [4]:
mode = "C.S"

secpar = 128
kappa = 80
log_q = 62
rep = 57
t = 256 * 3
T = t * rep
n_vdf = 14 

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 51.6,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel": 2,
    "log_beta_wit_inf": 0,
}

base = 2^7
ops_params = {
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = lambda base, n = True: [("bdecomp", {"base": base }), ("norm", {}), ("split", {"d": ops_params["d"]}), ("fold", {})] + ([("split", {"d": ops_params["d"]}), ("fold", {})] if n else [])

ops = small_loop + big_loop(2^9) + big_loop(2^9) + big_loop(2^9)+ [("finish", {})]


sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

print(f'VDF rounds T = {T}')


Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   666624 |  57 |        ( 13 | 13 )        |         (  0 | 13 )         | 36.24 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |   666624 |  72 |        ( 13 | 13 )        |         (  0 | 13 )         | 45.77 MB |      (205.6 KB | 205.6 KB)      |         (2^-120 | 2^-120)         
 split     |   333312 | 144 |        ( 13 | 49 )        |         (  0 | 41 )         | 45.77 MB |      (980.9 KB | 1.159 MB)      |         (2^-124 | 2^-120)         
 fold      |   333312 |  57 |        ( 21 | 43 )        |         (  9 | 37 )         | 181.2 MB |      (980.9 KB | 1.159 MB)      |         (2^-83  | 2^-83 )         
 split     |   166656 | 114 |        ( 21 | 36 )        |         (  9 | 33 )         | 181.2 MB |      (776.5 KB | 1.917 MB)      |         (2

In [5]:
mode = "B.L"

secpar = 96
kappa = 65
log_q = 62
rep = 57
t = 1024 * 3
T = t * rep
n_vdf = 14

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 54.6,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel": 2,
    "log_beta_wit_inf": 0,
}

base = 2^7
ops_params = {
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = lambda base, n = True: [("bdecomp", {"base": base }), ("norm", {}), ("split", {"d": ops_params["d"]}), ("fold", {})] + ([("split", {"d": ops_params["d"]}), ("fold", {})] if n else [])

ops = small_loop + big_loop(2^7) + big_loop(2^9) + big_loop(2^10) + big_loop(2^10) + [("finish", {})]


sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

print(f'VDF rounds T = {T}')


Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  2666496 |  57 |        ( 14 | 14 )        |         (  0 | 14 )         | 144.9 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |  2666496 |  73 |        ( 14 | 14 )        |         (  0 | 14 )         | 185.6 MB |      (188.6 KB | 188.6 KB)      |         (2^-120 | 2^-120)         
 split     |  1333248 | 146 |        ( 14 | 53 )        |         (  0 | 45 )         | 185.6 MB |      (857.5 KB | 1.022 MB)      |         (2^-124 | 2^-119)         
 fold      |  1333248 |  48 |        ( 22 | 47 )        |         (  9 | 41 )         | 610.3 MB |      (857.5 KB | 1.022 MB)      |         (2^-68  | 2^-68 )         
 split     |   666624 |  96 |        ( 22 | 40 )        |         (  9 | 37 )         | 610.3 MB |      (563.8 KB | 1.572 MB)      |         (2

In [6]:
mode = "B.M"

secpar = 96
kappa = 65
log_q = 62
rep = 57
t = 512 * 3
T = t * rep
n_vdf = 14

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 54.1,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel": 2,
    "log_beta_wit_inf": 0,
}

base = 2^7
ops_params = {
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = lambda base, n = True: [("bdecomp", {"base": base }), ("norm", {}), ("split", {"d": ops_params["d"]}), ("fold", {})] + ([("split", {"d": ops_params["d"]}), ("fold", {})] if n else [])

ops = small_loop + big_loop(2^8) + big_loop(2^9) + big_loop(2^10) + big_loop(2^9, false) + [("finish", {})]

sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

print(f'VDF rounds T = {T}')


Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  1333248 |  57 |        ( 14 | 14 )        |         (  0 | 14 )         | 72.47 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |  1333248 |  72 |        ( 14 | 14 )        |         (  0 | 14 )         | 91.55 MB |      (174.7 KB | 174.7 KB)      |         (2^-120 | 2^-120)         
 split     |   666624 | 144 |        ( 14 | 56*)        |         (  0 | 47 )         | 91.55 MB |      (832.6 KB | 1007. KB)      |         (2^-124 | 2^-120)         
 fold      |   666624 |  48 |        ( 22 | 49 )        |         (  9 | 44 )         | 305.2 MB |      (832.6 KB | 1007. KB)      |         (2^-68  | 2^-68 )         
 split     |   333312 |  96 |        ( 22 | 42 )        |         (  9 | 40 )         | 305.2 MB |      (555.1 KB | 1.526 MB)      |         (2

In [7]:
mode = "B.S"

secpar = 96
kappa = 65
log_q = 62
rep = 57
t = 256 * 3
T = t * rep
n_vdf = 14

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 51.3,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel": 2,
    "log_beta_wit_inf": 0,
}

base = 2^7
ops_params = {
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = lambda base, n = True: [("bdecomp", {"base": base }), ("norm", {}), ("split", {"d": ops_params["d"]}), ("fold", {})] + ([("split", {"d": ops_params["d"]}), ("fold", {})] if n else [])

ops = small_loop + big_loop(2^9) + big_loop(2^9) + big_loop(2^9) + [("finish", {})]


sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

print(f'VDF rounds T = {T}')


Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   666624 |  57 |        ( 13 | 13 )        |         (  0 | 13 )         | 36.24 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |   666624 |  72 |        ( 13 | 13 )        |         (  0 | 13 )         | 45.77 MB |      (159.3 KB | 159.3 KB)      |         (2^-120 | 2^-120)         
 split     |   333312 | 144 |        ( 13 | 49 )        |         (  0 | 41 )         | 45.77 MB |      (758.5 KB | 917.8 KB)      |         (2^-124 | 2^-120)         
 fold      |   333312 |  48 |        ( 21 | 43 )        |         (  9 | 37 )         | 152.6 MB |      (758.5 KB | 917.8 KB)      |         (2^-68  | 2^-68 )         
 split     |   166656 |  96 |        ( 21 | 36 )        |         (  9 | 33 )         | 152.6 MB |      (505.7 KB | 1.390 MB)      |         (2

In [8]:
mode = "A.XL"

secpar = 64
kappa = 50
log_q = 62
rep = 57
t = 2048 * 3
T = t * rep
n_vdf = 14

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 54.8,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel": 2,
    "log_beta_wit_inf": 0,
}

base = 2^7
ops_params = {
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = lambda base, n = True: [("bdecomp", {"base": base }), ("norm", {}), ("split", {"d": ops_params["d"]}), ("fold", {})] + ([("split", {"d": ops_params["d"]}), ("fold", {})] if n else [])

ops = small_loop + big_loop(2^6) + big_loop(2^8) + big_loop(2^10) + big_loop(2^10) + big_loop(2^10, false) + [("finish", {})]


sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

print(f'VDF rounds T = {T}')


Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  5332992 |  57 |        ( 15 | 15 )        |         (  0 | 15 )         | 289.9 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |  5332992 |  73 |        ( 15 | 15 )        |         (  0 | 15 )         | 371.3 MB |      (131.4 KB | 131.4 KB)      |         (2^-58  | 2^-58 )         
 split     |  2666496 | 146 |        ( 15 | 51 )        |         (  0 | 42 )         | 371.3 MB |      (596.7 KB | 728.1 KB)      |         (2^-62  | 2^-57 )         
 fold      |  2666496 |  38 |        ( 23 | 44 )        |         (  9 | 39 )         | 966.3 MB |      (596.7 KB | 728.1 KB)      |         (2^-53  | 2^-52 )         
 split     |  1333248 |  76 |        ( 23 | 37 )        |         (  9 | 35 )         | 966.3 MB |      (310.6 KB | 1.014 MB)      |         (2

In [9]:
mode = "A.L"

secpar = 64
kappa = 50
log_q = 62
rep = 57
t = 1024 * 3
T = t * rep
n_vdf = 14

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 54.3,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel": 2,
    "log_beta_wit_inf": 0,
}

base = 2^7
ops_params = {
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = lambda base, n = True: [("bdecomp", {"base": base }), ("norm", {}), ("split", {"d": ops_params["d"]}), ("fold", {})] + ([("split", {"d": ops_params["d"]}), ("fold", {})] if n else [])

ops = small_loop + big_loop(2^7) + big_loop(2^9) + big_loop(2^10) + big_loop(2^10) + [("finish", {})]


sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

print(f'VDF rounds T = {T}')


Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  2666496 |  57 |        ( 14 | 14 )        |         (  0 | 14 )         | 144.9 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |  2666496 |  73 |        ( 14 | 14 )        |         (  0 | 14 )         | 185.6 MB |      (129.5 KB | 129.5 KB)      |         (2^-58  | 2^-58 )         
 split     |  1333248 | 146 |        ( 14 | 53 )        |         (  0 | 45 )         | 185.6 MB |      (587.9 KB | 717.4 KB)      |         (2^-62  | 2^-57 )         
 fold      |  1333248 |  38 |        ( 22 | 46 )        |         (  9 | 41 )         | 483.2 MB |      (587.9 KB | 717.4 KB)      |         (2^-53  | 2^-52 )         
 split     |   666624 |  76 |        ( 22 | 40 )        |         (  9 | 37 )         | 483.2 MB |      (306.0 KB | 1023. KB)      |         (2

In [10]:
mode = "A.M"

secpar = 64
kappa = 50
log_q = 62
rep = 57
t = 512 * 3
T = t * rep
n_vdf = 14

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 53.8,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel": 2,
    "log_beta_wit_inf": 0,
}

base = 2^7
ops_params = {
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = lambda base, n = True: [("bdecomp", {"base": base }), ("norm", {}), ("split", {"d": ops_params["d"]}), ("fold", {})] + ([("split", {"d": ops_params["d"]}), ("fold", {})] if n else [])

ops = small_loop + big_loop(2^8) + big_loop(2^9) + big_loop(2^10) + big_loop(2^10, false) + [("finish", {})]


sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

print(f'VDF rounds T = {T}')


Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  1333248 |  57 |        ( 14 | 14 )        |         (  0 | 14 )         | 72.47 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |  1333248 |  72 |        ( 14 | 14 )        |         (  0 | 14 )         | 91.55 MB |      (120.2 KB | 120.2 KB)      |         (2^-58  | 2^-58 )         
 split     |   666624 | 144 |        ( 14 | 56*)        |         (  0 | 47 )         | 91.55 MB |      (571.1 KB | 691.3 KB)      |         (2^-62  | 2^-58 )         
 fold      |   666624 |  38 |        ( 22 | 49 )        |         (  9 | 44 )         | 241.6 MB |      (571.1 KB | 691.3 KB)      |         (2^-53  | 2^-53 )         
 split     |   333312 |  76 |        ( 22 | 42 )        |         (  9 | 40 )         | 241.6 MB |      (301.4 KB | 992.7 KB)      |         (2

In [11]:
mode = "A.S"

secpar = 64
kappa = 50
log_q = 62
rep = 57
t = 256 * 3
T = t * rep
n_vdf = 14

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 54,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel": 2,
    "log_beta_wit_inf": 0,
}

base = 2^7
ops_params = {
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = lambda base, n = True: [("bdecomp", {"base": base }), ("norm", {}), ("split", {"d": ops_params["d"]}), ("fold", {})] + ([("split", {"d": ops_params["d"]}), ("fold", {})] if n else [])

ops = small_loop + big_loop(2^9) + big_loop(2^10) + big_loop(2^10) + [("finish", {})]


sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

print(f'VDF rounds T = {T}')


Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   666624 |  57 |        ( 13 | 13 )        |         (  0 | 13 )         | 36.24 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |   666624 |  72 |        ( 13 | 13 )        |         (  0 | 13 )         | 45.77 MB |      (120.2 KB | 120.2 KB)      |         (2^-58  | 2^-58 )         
 split     |   333312 | 144 |        ( 13 | 49 )        |         (  0 | 41 )         | 45.77 MB |      (571.1 KB | 691.3 KB)      |         (2^-62  | 2^-58 )         
 fold      |   333312 |  38 |        ( 21 | 42 )        |         (  9 | 37 )         | 120.8 MB |      (571.1 KB | 691.3 KB)      |         (2^-53  | 2^-53 )         
 split     |   166656 |  76 |        ( 21 | 36 )        |         (  9 | 33 )         | 120.8 MB |      (301.4 KB | 992.7 KB)      |         (2

In [12]:
# mode = "C.L"

# secpar = 128
# kappa = 80
# log_q = 62
# rep = 57
# t = 64 * 22
# T = t * rep
# n_vdf = 128 

# outer_ring_params = {
#     "f": 24,
#     "log_beta_sis_2": 53.6,
#     "log_q": log_q,
#     "secpar_target": secpar,
#     "kappa_target": kappa
# }
# outer_ring = RingParam(**outer_ring_params)
# inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
# inner_ring_params = {
#     "f": 12,
#     "log_beta_sis_2": inner_log_beta_sis_2 + 1,
#     "log_q": 62,
# }
# inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
# assert n_vdf >= inner_ring.n_sis

# rel_params = {
#     "wdim": t * n_vdf * log_q,
#     "rep": rep,
#     "n_rel" : 2,
#     "log_beta_wit_inf": 0,
# }

# base = 2^7
# ops_params = {
#     "d": 2,
# }

# small_loop = [
#     ("norm", {}), 
#     # ("batch", {}), 
#     ("split", {"d": ops_params["d"]}), 
#     ("fold", {})
# ]

# big_loop = lambda base, n = True: [("bdecomp", {"base": base })] + ([("norm", {})] if n else [])   + [("split", {"d": ops_params["d"]}), ("fold", {})] +  [("split", {"d": ops_params["d"]}), ("fold", {})]

# ops = small_loop + big_loop(2^4) + big_loop(2^6) + big_loop(2^8) + big_loop(2^9) +  big_loop(2^7) + [("finish", {})]

# sim = Simulation(outer_ring_params, rel_params)
# sim.execute(ops)
# sim.extract()
# sim.show()

# print(f'VDF rounds T = {T}')
